FASE 3: Extracción de Detalles de Actores con Selenium

In [1]:
# Importar librerías para tratamiento de datos
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
from bs4 import BeautifulSoup

# Importar librerías para automatización de navegadores web con Selenium
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.

# Importar librerías para pausar la ejecución
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.


In [2]:
import pandas as pd
from selenium import webdriver 
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select 
from selenium.webdriver.common.by import By 
from selenium.common.exceptions import NoSuchElementException  #Este importé para manejar más errores.
from time import sleep
import os
import signal
import sys

Del csv con todas nuestras peliculas, nos quedamos solo con la columna de los ids (la comprobamos):

In [3]:
df_pelis = pd.read_csv("peliculas_api.csv")

In [4]:
df_pelis.keys()

Index(['Unnamed: 0', 'Tipo', 'Nombre', 'Año de Estreno', 'Mes de Estreno',
       'ID de la Película', 'Genero'],
      dtype='object')

In [4]:
id_IMDB= df_pelis["ID de la Película"]
id_IMDB[:5]

0    tt0146592
1    tt0154039
2    tt0162942
3    tt0230212
4    tt0326965
Name: ID de la Película, dtype: object

In [5]:
id_IMDB[0]


'tt0146592'

Comenzamos con SELENIUM!Por ahora, solo con una pelicula:

In [17]:
driver = webdriver.Chrome()
driver.get("https://www.imdb.com/title/tt0448115/")

sleep(3)
#aceptamos cookies
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

sleep(2)
#ampliamos pantalla
driver.maximize_window()

#hago clic en hamburguesa, para que me abra desplegable
driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button').click()
sleep(1)

#Voy a reparto y equipo
driver.find_element("xpath", "/html/body/div[4]/div[2]/div/div[2]/div/div[2]/div/div[1]/ul/li[2]/a").click()
sleep(2)

dicc_actors= {"Nombre actr" : [], 
              "conocide_por": [],
              "año_nacimiento":[],
              "profesión": [],
              "premios": []}

for i in range(2,12):
    driver.find_element("xpath", f'//*[@id="fullcredits_content"]/table[3]/tbody/tr[{i}]/td[2]/a').click()
    sleep(3)

    #Extraemos el nombre y apendeamos al diccionario.
    nombre_act = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div > h1 > span").text
    dicc_actors["Nombre actr"].append(nombre_act)

    sleep(5)

    #Extraemos las pelis por las que se le conoce y apendeamos al diccionario.
    conocide_por= driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-a6d4b6c0-0.bBRhdF > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(1) > div.ipc-primary-image-list-card__content > div.ipc-primary-image-list-card__content-top > a").text
    dicc_actors["conocide_por"].append(conocide_por)

    sleep(2)
    try:
        conocide_por2=driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-a6d4b6c0-0.bBRhdF > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(2) > div.ipc-primary-image-list-card__content > div.ipc-primary-image-list-card__content-top > a").text
        dicc_actors["conocide_por"].append(conocide_por2)
    except NoSuchElementException:
        pass

    #Extraemos la fecha de nacimiento y apendeamos al diccionario.
    sleep(3)
    fecha_nacimiento= driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-4.iDcoFI > div.sc-92625f35-6.gHKhNg > div.sc-92625f35-11.eyxQlE > section > aside > div > span:nth-child(2)").text
    dicc_actors["año_nacimiento"].append(fecha_nacimiento)

    sleep(2)
    profesion = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div > ul").text
    try:
        profesion_limpia = profesion.replace("\n", ", ")
        dicc_actors["profesión"].append(profesion_limpia)
    
    except NoSuchElementException:
        dicc_actors["profesión"].append(profesion)

    sleep(2)
    #por último, cogemos los premios, si los tiene y los apendeamos al diccionario.
    try:
        premios = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[2]/div/ul/li/div/ul/li/span').text   #"css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(8) > div > ul > li > div > ul > li > span").text
        dicc_actors["premios"].append(premios)

    except NoSuchElementException:
        dicc_actors["premios"].append("N/A")

    driver.back()



In [23]:
dicc_actors["profesión"]

['Reparto, Producción, Dirección',
 'Reparto, Producción',
 'Reparto, Banda sonora, Producción',
 'Reparto, Producción, Banda sonora',
 'Reparto, Producción, Guion',
 'Reparto, Producción, Dirección',
 'Reparto',
 'Reparto, Producción, Dirección',
 'Reparto, Dirección, Producción',
 'Reparto, Dirección, Producción']